# Content-Based Filtering

## 1. 개요

- Content-Based Filtering(콘텐츠 기반 필터링)은 **사용자가 선호하는 아이템의 콘텐츠 정보**(장르, 키워드, 설명 등)를 분석하여 유사한 아이템을 추천하는 방식

✅ 장점

- 사용자의 명시적 피드백 없이도 추천 가능
- 새로운 사용자가 적은 경우에도 적용 가능
    - cold start 문제 완화

❌ 단점
- 아이템의 콘텐츠 정보가 충분해야 추천 가능
- 기존 선호도와 유사한 아이템만 추천 (새로운 취향 발견 어려움)

## 2. 동작 원리

- 아이템(예: 영화)의 특징(Feature) 을 추출 (예: 장르, 줄거리 등)
- TF-IDF 벡터화를 통해 아이템을 수치화
- 코사인 유사도(Cosine Similarity) 를 계산하여 유사한 아이템 찾기
- 사용자가 좋아한 아이템과 유사한 아이템을 추천

## 3. 사용 분야
- 온라인 스트리밍 서비스: Netflix, YouTube, Spotify (영화/음악 추천)
- 전자상거래: Amazon, eBay (상품 추천)
- 뉴스 서비스: Google News, Flipboard (맞춤형 뉴스 추천)
- 교육 플랫폼: Coursera, Udemy (맞춤형 강의 추천)

## 4. 필요 수식
🔹 TF-IDF 벡터화

TF-IDF(Term Frequency-Inverse Document Frequency)는 다음과 같이 정의됩니다:

$$
TF_{t,d} = \frac{f_{t,d}}{\sum_{t' \in d} f_{t',d}}
$$

$$
IDF_t = \log \frac{N}{|\{d \in D : t \in d\}|}
$$

$$
TFIDF_{t,d} = TF_{t,d} \times IDF_t
$$

여기서:

TF : 문서 d에서 단어 t의 빈도

IDF : 전체 문서에서 단어 t의 희소성

N : 전체 문서 개수

🔹 코사인 유사도 (Cosine Similarity)

두 벡터 의 유사도는 다음과 같이 정의됩니다:

$$
Sim(A,B) = \frac{A \cdot B}{||A|| \times ||B||}
$$

이 공식은 두 벡터 간의 각도를 측정하여 얼마나 유사한지를 판단합니다.



In [47]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
# data set url

# dataset_url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
# dataset_path = 'ml-latest-small.zip'
# extracted_folder = 'ml-latest-small'

In [18]:
# movies.csv 파일 읽기
file_path = 'data/movies.csv'
movies = pd.read_csv(file_path)
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [23]:
movies['genres'].value_counts(dropna=False)

genres
Drama                                        1053
Comedy                                        946
Comedy|Drama                                  435
Comedy|Romance                                363
Drama|Romance                                 349
                                             ... 
Children|Drama|Musical                          1
Adventure|Drama|Horror|Mystery|Thriller         1
Adventure|Children|Comedy|Fantasy|Mystery       1
Adventure|Animation|Children|Western            1
Comedy|Mystery|Romance|Thriller                 1
Name: count, Length: 951, dtype: int64

In [31]:
# TF-IDF 벡터화
tfidf = TfidfVectorizer(stop_words="english")  # 불용어 제거
tfidf_matrix = tfidf.fit_transform(movies['genres'])  # TF-IDF 변환
tfidf_matrix

<9742x23 sparse matrix of type '<class 'numpy.float64'>'
	with 23185 stored elements in Compressed Sparse Row format>

In [33]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.81357774, 0.15276924, ..., 0.        , 0.4210373 ,
        0.26758648],
       [0.81357774, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15276924, 0.        , 1.        , ..., 0.        , 0.        ,
        0.57091541],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.4210373 , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.26758648, 0.        , 0.57091541, ..., 0.        , 0.        ,
        1.        ]])

In [35]:
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()


In [44]:
def recommend_movies(title, cosine_sim=cosine_sim, num_movies=10):
    if title not in indices:
        print(f"'{title}' 제목을 찾을 수 없습니다.")
        return []
    
    idx = indices[title]  # 입력 영화의 인덱스 찾기
    sim_scores = list(enumerate(cosine_sim[idx]))  # 모든 영화와의 유사도 리스트
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # 유사도 기준 정렬
    sim_scores = sim_scores[1:num_movies+1]  # 자기 자신 제외하고 상위 num_movies개 선택

    movie_indices = [i[0] for i in sim_scores]  # 영화 인덱스 가져오기
    
    return movies.iloc[movie_indices][['title']]  # 추천 영화 목록 반환

In [45]:
print(recommend_movies("Toy Story (1995)"))


                                                  title
1706                                        Antz (1998)
2355                                 Toy Story 2 (1999)
2809     Adventures of Rocky and Bullwinkle, The (2000)
3000                   Emperor's New Groove, The (2000)
3568                              Monsters, Inc. (2001)
6194                                   Wild, The (2006)
6486                             Shrek the Third (2007)
6948                     Tale of Despereaux, The (2008)
7760  Asterix and the Vikings (Astérix et les Viking...
8219                                       Turbo (2013)
